# PGT KEP 

### Reproduce the computational tool, ETSeq

In [1]:
###Setup###
import pandas as pd
import numpy as np
import math
import itertools

In [2]:
###input###
temp_df = pd.read_excel('/rds/homes/e/exk986/ws/MSc_PGTKEP_IndividualRepo/Data/templateSeqs.xlsx')
temp_df = temp_df.drop(temp_df[temp_df['P90'] == 0.0].index)
temp_df

,Template Sequence,TM1,TM2,# Bonds,Results,P90,N10,DIFF
0,CCTACGACTGAACAGACTCTCCTACGACTG,43,15,6,CI,12.6,16.6,4.0
1,CCTACGACTTAACAGACTCTCCTACGACTT,39,-25,2,CI,16.3,21.4,5.1
2,CCTACGACGGAACAGACTCTCCTACGACGG,48,-25,2,CI,15.4,16.5,1.2
3,CCTACGAGTGAACAGACTCTCCTACGAGTG,43,-25,2,CIII,17.8,17.8,0.1
4,CCTACGGCTGAACAGACTCTCCTACGGCTG,48,2,6,CIII,12.5,12.0,-0.5
...,...,...,...,...,...,...,...,...
376,TAACTACGGCGGTTGACTCTTAACTACGGC,40,19,4,CII,26.5,19.8,-6.7
378,ATTAGCCAGCCGATGACTCTATTAGCCAGC,41,-40,2,CIII,11.5,12.4,0.8
379,GTAGGCGATGATGAGACTCTGTAGGCGATG,44,-34,2,CI,9.0,11.1,2.2
382,ACCAAAAGGCTAGCGACTCTACCAAAAGGC,43,14,6,CII,30.3,27.0,-3.2


In [3]:
# Data Preprocessing
# Remove 77 sequences from whole dataset
def gen_valid_df(df):
    ci_df = pd.concat([df[df['Results'].eq('CI')], df[df['Results'].str.contains('CI,')]])
    cii_df = pd.concat([df[df['Results'].eq('CII')], df[df['Results'].str.contains('CII,')]])
    valid_df = pd.concat([ci_df, cii_df])
    return valid_df, ci_df, cii_df

In [4]:
# Identify the template sequences labeled as Class I OR Class II
valid_df, ci_df, cii_df = gen_valid_df(temp_df)

In [5]:
valid_df # The total data set; combination of Class I AND Class II

,Template Sequence,TM1,TM2,# Bonds,Results,P90,N10,DIFF
0,CCTACGACTGAACAGACTCTCCTACGACTG,43,15,6,CI,12.6,16.6,4.0
1,CCTACGACTTAACAGACTCTCCTACGACTT,39,-25,2,CI,16.3,21.4,5.1
2,CCTACGACGGAACAGACTCTCCTACGACGG,48,-25,2,CI,15.4,16.5,1.2
5,CCTACTACTGAACAGACTCTCCTACTACTG,36,15,6,CI,18.8,23.8,5.0
7,CCTGCGACTGAACAGACTCTCCTGCGACTG,49,15,6,CI,11.9,13.2,1.3
...,...,...,...,...,...,...,...,...
251,GCCGACAAGACTTTGACTCTGCCGACAAGA,47,-31,6,"CII,NS",17.3,15.8,-1.5
259,GCCGCGATAACGAGGACTCTGCCGCGATAA,47,15,4,"CII,LA",44.2,41.0,-3.2
294,CGATGATAACATCCGACTCTCGATGATAAC,35,-8,4,"CII,LA",45.1,40.1,-5.0
343,TCAGAAGACTAACCGACTCTTCAGAAGACT,37,21,8,"CII,LA",35.4,27.8,-7.6


In [6]:
valid_df = valid_df.reset_index(drop=True)

In [7]:
seqs = [seq for seq in valid_df.iloc[:,0]] 
len(seqs)

204

##### PWM

In [8]:
### Original PWMs in public source code ###
pwm_diff = [[0.470003629245736,0.628608659422374,0.191055236762709,0.969400557188104,0.887303195000903,0.594707107746693,1.12846525181779,1.09861228866811,0.810930216216329,1.09861228866811,0,0,0.550046336919272,0,0.470003629245736,0.628608659422374,0.191055236762709,0.969400557188104,0.887303195000903,0.594707107746693,1.12846525181779,1.09861228866811,0.810930216216329,1.09861228866811], [0.287682072451781,0.693147180559945,0,0.231801614057324,1.04145387482816,0.371563556432483,0.961411167154625,0.587786664902119,0.204794412646013,0,0.559615787935423,0.0953101798043249,0.122602322092332,0.424883193965266,0.287682072451781,0.693147180559945,0,0.231801614057324,1.04145387482816,0.371563556432483,0.961411167154625,0.587786664902119,0.204794412646013,0], [0,0.405465108108164,0,0.476924072090309,0.481838086892738,0.594707107746693,0.390866308687012,1.01856958099457,0.300104592450338,0.916290731874155,0.287682072451781,0.146603474191875,0.424883193965266,0.367724780125317,0,0.405465108108164,0,0.476924072090309,0.481838086892738,0.594707107746693,0.390866308687012,1.01856958099457,0.300104592450338,0.916290731874155], [0,0,0.362905493689368,0,0,0,0,0,0,0.0339015516756814,0.559615787935423,0.0953101798043249,0,0.262364264467491,0,0,0.362905493689368,0,0,0,0,0,0,0.0339015516756814]];
pwm_p90 = [[0.847297860387204,0.998528830111127,1.14513230430300,1.01160091167848,0.641853886172395,0.546543706368070,0.737598943130779,0.650587566141149,1.99243016469021,1.09861228866811,0,0.0606246218164348,0.348306694268216,0,0.847297860387204,0.998528830111127,1.14513230430300,1.01160091167848,0.641853886172395,0.546543706368070,0.737598943130779,0.650587566141149,1.99243016469021,1.09861228866811],[0.271933715483642,0,0.200670695462151,0.606135803570316,0.305381649551182,0.236388778064230,0.832909122935104,0.427444014826940,0.382992252256106,0,0.117783035656383,0,0,0.133531392624523,0.271933715483642,0,0.200670695462151,0.606135803570316,0.305381649551182,0.236388778064230,0.832909122935104,0.427444014826940,0.382992252256106,0],[0.336472236621213,0.0540672212702758,0,0.146603474191875,0.0540672212702758,0.171850256926659,0.302280871872934,0.737598943130779,0.0465200156348929,0.538996500732687,0.117783035656383,0.348306694268216,0.0606246218164348,0.0645385211375712,0.336472236621213,0.0540672212702758,0,0.146603474191875,0.0540672212702758,0.171850256926659,0.302280871872934,0.737598943130779,0.0465200156348929,0.538996500732687],[0,0.111225635110224,0.451985123743057,0,0,0,0,0,0,0.470003629245736,0.492476485097794,0.0606246218164348,0.0606246218164348,0,0,0.111225635110224,0.451985123743057,0,0,0,0,0,0,0.470003629245736]];

In [9]:
#p90_threshold = valid_df.loc[:,'P90'].quantile(0.30)
#p90_template = valid_df[valid_df['P90'] <= p90_threshold]
#p90_template

In [10]:
#[seq for seq in p90_template.iloc[:,0]]

In [11]:
# PWM based method
# Step01. Generate PWMs
def gen_partial_seq(seq): # remove the sequences in postion 15 to 20; Nicking enzyme recognition site + additional 'T'
    return seq[seq.find('GACTC')-14:seq.find('GACTC')]+seq[seq.find('GACTC')-14:seq.find('GACTC')-4]

def gen_pwm(seqs): # generate basic PWM
    mat1 = [[0 for col in range(24)] for row in range(4)]
    pwm = [[0 for col in range(24)] for row in range(4)]
    
    for seq in seqs:
        partial_seq = gen_partial_seq(seq)
        if partial_seq == '':
            continue
        for i in range(24):
            if partial_seq[i] == 'A':
                mat1[0][i] += 1
            if partial_seq[i] == 'C':
                mat1[1][i] += 1
            if partial_seq[i] == 'G':
                mat1[2][i] += 1
            if partial_seq[i] == 'T':
                mat1[3][i] += 1
    
    mat1 = np.array(mat1)
    
    for col in range(24):
        max_num = max(mat1.T[col])
        for row in range(4):
            pwm[row][col] = math.log(max_num / mat1[row][col])
            
    return pwm

def gen_p90_pwm(df): # generate PWM related to P90 score   
    p90_threshold = df.loc[:,'P90'].quantile(0.30)
    p90_template = df[df['P90'] <= p90_threshold]
    seqs = [seq for seq in p90_template.iloc[:,0]]

    return gen_pwm(seqs)

def gen_diff_pwm(df): # generate PWM related to Diff score    
    diff_threshold = df.loc[:,'DIFF'].quantile(0.60)
    diff_template = df[df['DIFF'] >= diff_threshold]
    seqs = [seq for seq in diff_template.iloc[:,0]]

    return gen_pwm(seqs)

In [12]:
# Generate the PWM using the templates in lowest 30% in P90 spectrum
p90_pwm = gen_p90_pwm(valid_df)
p90_pwm

[[0.7985076962177716,
  0.9985288301111273,
  1.0116009116784799,
  1.0116009116784799,
  0.6418538861723947,
  0.45953232937844013,
  0.6505875661411494,
  0.6505875661411494,
  2.03688192726104,
  1.2321436812926323,
  0.0,
  0.06062462181643484,
  0.4054651081081644,
  0.0,
  0.7985076962177716,
  0.9985288301111273,
  1.0116009116784799,
  1.0116009116784799,
  0.6418538861723947,
  0.45953232937844013,
  0.6505875661411494,
  0.6505875661411494,
  2.03688192726104,
  1.2321436812926323],
 [0.0,
  0.1112256351102244,
  0.4519851237430572,
  0.0,
  0.05406722127027579,
  0.0,
  0.0,
  0.0,
  0.0,
  0.4054651081081644,
  0.6418538861723947,
  0.06062462181643484,
  0.1823215567939546,
  0.06062462181643484,
  0.0,
  0.1112256351102244,
  0.4519851237430572,
  0.0,
  0.05406722127027579,
  0.0,
  0.0,
  0.0,
  0.0,
  0.4054651081081644],
 [0.28768207245178085,
  0.05406722127027579,
  0.0,
  0.14660347419187544,
  0.0,
  0.17185025692665923,
  0.3629054936893685,
  0.7375989431307791,

In [13]:
# Generate the PWM using the templates in highest 40% in Diff spectrum
diff_pwm = gen_diff_pwm(valid_df)
diff_pwm

[[0.47000362924573563,
  0.5596157879354227,
  0.28768207245178085,
  0.8979415932059586,
  0.8574502318512216,
  0.6931471805599453,
  1.0986122886681098,
  1.0185695809945732,
  0.7731898882334817,
  1.236762627148927,
  0.0,
  0.0,
  0.5108256237659907,
  0.0,
  0.47000362924573563,
  0.5596157879354227,
  0.28768207245178085,
  0.8979415932059586,
  0.8574502318512216,
  0.6931471805599453,
  1.0986122886681098,
  1.0185695809945732,
  0.7731898882334817,
  1.236762627148927],
 [0.0425596144187959,
  0.0,
  0.4054651081081644,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.10178269430994238,
  0.6241543090729939,
  0.09531017980432493,
  0.0,
  0.30010459245033816,
  0.0425596144187959,
  0.0,
  0.4054651081081644,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.10178269430994238],
 [0.0,
  0.3364722366212129,
  0.0,
  0.3513978868378886,
  0.4519851237430572,
  0.6286086594223741,
  0.4054651081081644,
  1.0185695809945732,
  0.3136575588550416,
  0.949080554697146,
  0.2876820724517

In [14]:
# Comparison between initial PWMs and newly formed PWMs
pwm_p90 == p90_pwm

False

In [15]:
pwm_diff == diff_pwm

False

In [16]:
# PWM based method
# Step02. Calculate the P90/Diff score for the template by generated PWMs
def pwm_pred(pwm=[], seq=''):
    mat_1 = [[0 for col in range(24)] for row in range(4)]
    mat_2 = [[0 for col in range(24)] for row in range(4)]
    mat_score = [0 for x in range(24)]
    score = 0

    for i in range(24):
        if seq[i] == 'A':
            mat_1[0][i] = 1
        if seq[i] == 'C':
            mat_1[1][i] = 1
        if seq[i] == 'G':
            mat_1[2][i] = 1
        if seq[i] == 'T':
            mat_1[3][i] = 1
            
    for i in range(24):
        for j in range(4):
            mat_2[j][i] = mat_1[j][i]*pwm[j][i]
            # print(mat_1[j][i])
            
    for i in range(24):
        for j in range(4):
            mat_score[i] = mat_score[i] + mat_2[j][i]
    
    score = sum(mat_score)
    
    return score

In [17]:
# PWM based method
# Step03. Classify either Class I or II
def pwm_class(P90_score, Diff_score):
    if Diff_score > -0.8510*P90_score+15.8764 :
        return 'CII'
    else:
        return 'CI'

In [18]:
p90s = []
diffs = []
classes = []
for seq in seqs:
    pwm_scores = {}
    
    i = seqs.index(seq)
    p90_score = pwm_pred(p90_pwm, seq)
    diff_score = pwm_pred(diff_pwm, seq)
    pwm_cls = pwm_class(p90_score, diff_score)
    
    p90s.append(p90_score)
    diffs.append(diff_score)
    classes.append(pwm_cls)

pwm_scores['template'] = seqs
pwm_scores['P90'] = p90s
pwm_scores['Diff'] = diffs
pwm_scores['class'] = classes

In [19]:
pwm_results = pd.DataFrame(pwm_scores)
pwm_results.transpose()
pwm_results

,template,P90,Diff,class
0,CCTACGACTGAACAGACTCTCCTACGACTG,7.829916,6.767963,CI
1,CCTACGACTTAACAGACTCTCCTACGACTT,7.290919,5.818883,CI
2,CCTACGACGGAACAGACTCTCCTACGACGG,7.542234,6.959019,CI
3,CCTACTACTGAACAGACTCTCCTACTACTG,7.963447,6.596113,CI
4,CCTGCGACTGAACAGACTCTCCTGCGACTG,6.271771,5.933738,CI
...,...,...,...,...
199,GCCGACAAGACTTTGACTCTGCCGACAAGA,8.754124,9.544737,CII
200,GCCGCGATAACGAGGACTCTGCCGCGATAA,10.765935,9.922185,CII
201,CGATGATAACATCCGACTCTCGATGATAAC,11.778388,8.492537,CII
202,TCAGAAGACTAACCGACTCTTCAGAAGACT,9.962967,8.363401,CII


In [20]:
actual_results = valid_df['Results']
pwm_results = pwm_results.join(actual_results)
pwm_results

,template,P90,Diff,class,Results
0,CCTACGACTGAACAGACTCTCCTACGACTG,7.829916,6.767963,CI,CI
1,CCTACGACTTAACAGACTCTCCTACGACTT,7.290919,5.818883,CI,CI
2,CCTACGACGGAACAGACTCTCCTACGACGG,7.542234,6.959019,CI,CI
3,CCTACTACTGAACAGACTCTCCTACTACTG,7.963447,6.596113,CI,CI
4,CCTGCGACTGAACAGACTCTCCTGCGACTG,6.271771,5.933738,CI,CI
...,...,...,...,...,...
199,GCCGACAAGACTTTGACTCTGCCGACAAGA,8.754124,9.544737,CII,"CII,NS"
200,GCCGCGATAACGAGGACTCTGCCGCGATAA,10.765935,9.922185,CII,"CII,LA"
201,CGATGATAACATCCGACTCTCGATGATAAC,11.778388,8.492537,CII,"CII,LA"
202,TCAGAAGACTAACCGACTCTTCAGAAGACT,9.962967,8.363401,CII,"CII,LA"


In [21]:
pwm_results.to_excel(r'/rds/homes/e/exk986/ws/MSc_PGTKEP_IndividualRepo/Reproduce_ETSeq/pwm_results.xlsx', index=False)

### Naive Bayes

In [22]:
### Original sequence motifs in public source code ###
orig_features = ['C-8','A-9','A-10','T-10','GA-8','C-7','A-1','G-8','C-6','GA-9','G-3','CC-7','AG-9','CT-9','C-10','GG-3','CG-10','CG-6','A-3','CC-8','AG-1','A-6','C-9','GA-5','AC-7','T-9','AC-10','AAC-11','A-8','CGA-7','TT-9','AG-8','TG-2','TC-6','T-2','AC-12','TA-10','TA-7','AGT-1','CCT-8','CC-1','CT-7','CA-2','GAC-9','CT-8','TGG-3','AA-11','AG-12','GG-1','CG-7','G-7','AA-10','GAC-5','GG-4','AG-6','A-7','G-2','CTC-6','GT-8','GAG-8','AC-3','AA-3','AA-1','CAA-9','GC-5','GT-7','TAC-7','ACT-10','GA-2','GGAC-4','CCG-5','T-1','CTG-9','ATC-4','GTT-8','A-12','AACA-11','CGG-5','GGA-1','A-2','TG-8','CAA-10','ACA-3','TGG-2','TGA-7','TAG-11','GAC-2','CC-9','AT-1','ACTC-10','GAG-5','CCT-7','CTC-9','TC-4','AA-8','CCTC-8','ACC-12','ACCG-12','C-11','T-8','CAG-7','CCGC-1','CGA-8','CCG-1','TGGA-3','GAC-6','CG-3','CGAC-5','ACC-6','TA-9','ACG-9','GACC-5','TAG-10','TTAG-9','GGT-3','GTTA-8','GG-2','GGA-7','TCT-6','TC-10','CT-11','CA-4','GT-4','TCG-4','G-11','A-4','CGT-4','ATGT-9','TCTG-8','TA-2','TCT-8','AAAC-10','TGC-4','CCT-4','G-9','CCC-8','C-4','T-5','GGA-4','GCC-3','ACGA-6','GAT-5','AC-9','CT-10','GAA-8','GT-1','TG-10','GTT-11','TTG-9','TG-6','CG-8','CTC-5','TAG-7','TA-1','CTA-10','TAA-10','CGC-6','GTA-1','AGTG-12','AAG-1','GGAC-1','CGCC-10','TC-12','GTA-6','AGT-12','CTG-8','CA-7','TC-9','ACG-5','AAC-6','AGA-10','AA-4','A-5','CT-1','GTG-7','GGT-2','GT-5','TAC-11','AC-6','TGA-8','GTG-4','CGAA-7','AT-6','ACG-6','C-13','GT-9','ATG-1','AAA-8','C-1','AG-2','AGA-2','TTA-9','CTC-11','TC-5','GAC-8','AT-4','CCC-7','ACA-1','AGA-9','AG-4','GCC-7','AGA-6','GAA-7','TCC-4','TAC-2','GTG-12','GGG-10','GC-7','GTA-9','TTC-12','AGA-8','GT-3','CTCC-9','GACT-9','TTCG-12','CCGC-5','TGGA-6','GTGG-12','GTAA-9','G-5','CGC-2','CT-5','CTA-5','CTG-7','TAAT-10','CTAC-4','CGC-10','AAC-10','AGAA-8','TT-4','TA-11','GCTA-9','TTT-4','GGG-6','CCTG-7','CGAG-7','TTGT-9','TGGG-2','CC-13','CG-5','CCGA-13','CCG-13','TGG-8','AAG-9','CGAC-14','CGTG-12','CGAC-8','C-14','CGT-12','CTGG-7','TGT-6','CGA-14','CG-14','TC-7','ATC-3','GCC-11','AG-10','GTG-5','GCT-11','TCA-6','TCA-5','GCA-4','TCC-10','GAAG-8','TCCT-7','ACT-3','CGG-10','GGCG-8','TAG-1','AAC-8','ACC-8','TT-10','ACAA-3','CG-2','GGT-7','AT-9','GGC-8','AT-10','CTT-11','TGA-4','TACG-5','CG-4','ACGA-7','CGA-1','AGG-6','AC-2','TGG-9','GCA-6','TACG-12','ACCT-7','GCA-8','TCTG-12','GCT-9','ATG-9','CCGC-9','TT-12','TCT-12','ACG-7','CTC-3','GCG-7','TTA-11','GCG-2','TAC-12','AGG-1','AC-1','GTA-7','CCAG-12','TAGC-11','G-10','CCTC-4','TAC-1','CCA-12','TG-7','GG-6','ACT-9','CGT-8','CCC-6','ATC-5','CTAG-10','GC-2','CGC-1','GGG-3','ATG-5','TACC-1','GGT-6','ATGA-1','CA-9','AAA-11','CC-5','GC-3','AA-13','TAGA-1','GCG-5','GCG-11','GA-11','GCG-9','TCAT-6','GCT-7','GCGA-5','CTG-6','ATA-4','TGC-6','AGAT-9','CGT-6','GGA-11','TAC-5','AAGA-13','ATC-11','GAAC-10','GCT-5','GAT-1','CAC-7','AAG-13','ACC-3','GATG-1','ATAC-1','ACA-9','ATG-2','GGG-1','CT-12','CTA-11','CAC-4','AGC-9','TTT-5','GG-8','TA-8','ACTA-10','AA-5','TGA-6','ATA-1','ACG-1','TCGC-1','CAT-2','CA-11','AC-8','TGCT-4','CAT-11','TCC-7','ACG-3','AAC-9','TACA-11','AGC-8','GAA-10','GTT-2','GAT-3','GTC-9','GCTG-6','ACC-2','ACC-7','CAA-4','CCT-10','CCG-9','GCC-9','CCG-8','AACC-11','CCCG-6','CTC-4','GT-6','CCG-12']    

len(orig_features) # num. of orig_features

399

In [23]:
# Find the sequence motifs which is used for position motif
nucleotides = ['A', 'G', 'T', 'C'] # define the characters to use
motifs = []

variable_pos = 14 # position 1 to 14

# Generate all permutations of strings up to length 4
for r in range(1, 5):  # lengths from 1 to 4
    for perm in itertools.product(nucleotides, repeat=r):
        subseq = ''.join(perm)
        
        for p in range(variable_pos):
            #print(subseq+'-'+str(p+1))
            motifs.append(subseq+'-'+str(p+1))

In [24]:
# Naive Bayes based method
# Step01. Generate sequence motifs
def cmp(a, b):
    return (a > b) - (a < b)

motif_counts = {} 

for m in motifs: # m = 'CTAT-11'
    mer = m.index('-') # mer = 4
    subseq = m[0:mer] # subseq = 'CTAT'
    start = m[mer+1:] # start = '11'
    
    for seq in seqs:
        if cmp(seq[int(start)-1:int(start)-1+len(subseq)],subseq)==0:
            motif_counts[m] = motif_counts.get(m,0) + 1

# store the motifs which is represented >== 3 times            
eff_motifs = [motif for motif in motif_counts if motif_counts[motif] >= 3]

In [25]:
#motif_counts #dictonary; motif:num. of appearance
#eff_motifs # lists; sequence motifs >= 3 times
len(eff_motifs)

956

In [26]:
# Naive Bayes based method
# Step02. Build position motif count matrix
def bayes_pred(seqs, class_name, eff_motifs):
    result = {} # position motif count matrix
    
    for motif in eff_motifs: 
        presence = []
        
        for seq in seqs:
            mer = motif.index('-')
            subseq = motif[0:mer]
            start = motif[mer+1:]
            
            if cmp(seq[int(start)-1:int(start)-1+len(subseq)],subseq)==0:
                presence.append(1)
                
            else:
                presence.append(0)  
                
        result['class'] = class_name
        result[motif] = presence
        
    return pd.DataFrame(result)

In [27]:
ci_seqs = [seq for seq in ci_df.iloc[:,0]]
ci_data = bayes_pred(ci_seqs, 1, eff_motifs)
ci_data

,class,A-1,A-2,A-3,A-4,A-5,A-6,A-7,A-8,A-9,...,CCTG-1,CCTG-4,CCTG-5,CCTG-7,CCTC-4,CCTC-5,CCTC-8,CCCG-3,CCCG-6,CCCG-12
0,1,0,0,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,1,0,0,...,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97,1,0,0,0,0,0,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
98,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
99,1,0,1,0,0,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
100,1,0,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [28]:
cii_seqs = [seq for seq in cii_df.iloc[:,0]]
cii_data = bayes_pred(cii_seqs, 0, eff_motifs)
cii_data

,class,A-1,A-2,A-3,A-4,A-5,A-6,A-7,A-8,A-9,...,CCTG-1,CCTG-4,CCTG-5,CCTG-7,CCTC-4,CCTC-5,CCTC-8,CCCG-3,CCCG-6,CCCG-12
0,0,0,0,1,0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,0,1,0,0,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,0,1,0,0,0,0,0,0,1,1,...,0,0,0,0,0,0,0,0,0,0
3,0,0,1,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,1,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97,0,0,0,0,0,1,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
98,0,0,0,0,0,0,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
99,0,0,0,1,0,0,1,0,1,1,...,0,0,0,0,0,0,0,0,0,0
100,0,0,0,1,0,1,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [29]:
pmcm_data = pd.concat([ci_data,cii_data])
pmcm_data = pmcm_data.reset_index(drop=True)
pmcm_data

,class,A-1,A-2,A-3,A-4,A-5,A-6,A-7,A-8,A-9,...,CCTG-1,CCTG-4,CCTG-5,CCTG-7,CCTC-4,CCTC-5,CCTC-8,CCCG-3,CCCG-6,CCCG-12
0,1,0,0,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,1,0,0,...,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199,0,0,0,0,0,1,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
200,0,0,0,0,0,0,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
201,0,0,0,1,0,0,1,0,1,1,...,0,0,0,0,0,0,0,0,0,0
202,0,0,0,1,0,1,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [30]:
pmcm_data.to_excel(r'/rds/homes/e/exk986/ws/MSc_PGTKEP_IndividualRepo/Reproduce_ETSeq/pmcm_results.xlsx', index=False)

In [31]:
# Naive Bayes based method
# Step03. Classify with Naïve Bayes classification

### Setup ###
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.metrics import classification_report

In [32]:
def evaluate(conf_mat, n_classes=2):
    for i in range(n_classes):
        tp = conf_mat[i, i]
        fn = sum(conf_mat[i, :]) - tp
        fp = sum(conf_mat[:, i]) - tp
        tn = sum(sum(conf_mat)) - tp - fn - fp
        
        tpr = tp / (tp + fn) #sensitivity
        tnr = tn / (tn + fp) #specificity
    return tpr, tnr

In [33]:
X = pmcm_data.iloc[:, 1:] 
y = pmcm_data.iloc[:,0] # Class

sens = []
specs = []

skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=28)

for train_index, test_index in skf.split(X, y): # cross-validation
    x_train_fold, x_test_fold = X.iloc[train_index], X.iloc[test_index]
    y_train_fold, y_test_fold = y.iloc[train_index], y.iloc[test_index]
    
    nb = MultinomialNB(alpha=1) # Naive Bayes classification
    nb.fit(x_train_fold, y_train_fold)
    
    sensitivity, specificity = evaluate(confusion_matrix(y_test_fold, nb.predict(x_test_fold))) #evaluation
    sens.append(sensitivity)
    specs.append(specificity)

In [34]:
sum(sens)/len(sens)

0.6863636363636363

In [35]:
sum(specs)/len(specs)

0.7081818181818182

In [36]:
print(classification_report(y_test_fold, nb.predict(x_test_fold)))

              precision    recall  f1-score   support

           0       0.69      0.90      0.78        10
           1       0.86      0.60      0.71        10

    accuracy                           0.75        20
   macro avg       0.77      0.75      0.74        20
weighted avg       0.77      0.75      0.74        20

